# Philita
### Tidy(er) version 

### Imports

In [5]:
import threading 
import sys
import pandas as pd
import requests
import time
import psutil
import subprocess
import numpy as np
import re
import math

### Functions and Classes
>#### Custom utilities for use in the main execution path 

In [6]:
def routesearchfunc(route_id_func, T_mod_func):
    "Identifies the shared index between the specified time and routeId"
    
    # Use that index to find it, effectively in reverse. 
    Id_location0 = myseries_routeId.index[myseries_routeId.tolist().index(route_id_func)]

    # Add a time index bias
    time_add_count = int(T_mod_func[0:2])*2
    if T_mod_func[3:5] == "30":
        time_add_count += 1

    Id_location_timeadd = Id_location0 + time_add_count

    return(Id_location_timeadd)

In [7]:
# Journey class
class Journey:
    "Stores journey information"
    
    StartLat = None
    StartLon = None
    EndLat = None
    EndLon = None
    StartTime = None
    PopAmount = None 
    BusyStops = None
    ban_stops = None

journeys = Journey()

In [8]:
def stopsearchfunc(stop_id_func, T_mod_func):
    "Identifies the shared index between the specified time and stopId"
    
    # Use that index to find it, effectively in reverse. 
    Id_location0 = myseries_stopId.index[myseries_stopId.tolist().index(stop_id_func)]

    # Add a time index bias
    time_add_count = int(T_mod_func[0:2])*2
    if T_mod_func[3:5] == "30":
        time_add_count += 1

    Id_location_timeadd = Id_location0 + time_add_count

    return(Id_location_timeadd)

In [9]:
def TimeMod(T, s1):
    "Modify the current time index to avoid xx:15:xx and xx:45:xx. Instead it rounds to the next interval."
    
    if T[s1] == "15":
        T_mod = T[0:3]+"30"+T[5:8]
    elif T[s1] == "45":
        if T == "23:45:00":
            T_mod = "00:00:00"
        else:
            T_mod = T[0:3]+"00"+T[5:8]
    else:
        T_mod = T
    
    return(T_mod)

In [10]:
def TimeModPrev(T_mod, s1):
    "Modify the current time index to be the previous one"
    
    if T_mod[s1] == "30":
        T_mod_prev = T_mod[0:3]+"00"+T_mod[5:8]

    elif T_mod[0] == "0":
        T_mod_prev = "0"+str(int(T_mod[1])-1)+":"+"30"+T_mod[5:8]
        
    elif T_mod[s1] == "00":
        T_mod_prev = str(int(T_mod[0:2])-1)+":"+"30"+T_mod[5:8]
                
    elif T_mod == "01:00:00":
        T_mod_prev == "00:30:00"
        
    else:
        T_mod_prev = T_mod  
        
    return(T_mod_prev)

>#### The multithreading-compatible function which communicates with the OTP and does the looping.  

In [11]:
def StopCapPhilitaMain_thread(k1, T, port_no, StopCapacity, RouteCapacity):
#     global StopCapacity
#     global RouteCapacity
    global stopIds_holder
    global routeIds_holder
    global Philita_count
    global journeys
    unclean = PopInput.loc["{}".format(T),"{}".format(k1)]
    if unclean is not np.nan:
        unclean_split = unclean.split()
        
        # Make the start and end locations split

        threadLock.acquire()
        journeys.StartLat = re.sub("[^\d\.\-]", "", "{}".format(unclean_split[0]))
        journeys.StartLon = re.sub("[^\d\.\-]", "", "{}".format(unclean_split[1]))
        journeys.EndLat = re.sub("[^\d\.\-]", "", "{}".format(unclean_split[2]))
        journeys.EndLon = re.sub("[^\d\.\-]", "", "{}".format(unclean_split[3]))
        threadLock.release()

        print("Journey from port "+str(port_no)+" with parameters "+ journeys.StartLat, journeys.StartLon +", "+ journeys.EndLat, journeys.EndLon)
        # Modified Time
        T_mod = TimeMod(T, s1)
        T_mod_prev = TimeModPrev(T_mod, s1)
        
        
        # Create the unpreferred routes index 
        unpreferredRoutes = []
        index_count = -1
        
        
        
        # COMMENTED OUT FOR NON-PHILITA
        # Append to unpreferred routes if business of time indexed is greater than the threshold
        for row in RouteCapacity.loc[:,"business"]:
            index_count += 1
            
            if row >= route_b_threshold:
                if RouteCapacity.loc[index_count,"time"] == T_mod or RouteCapacity.loc[index_count,"time"] == T_mod_prev:
                    unprefRouteHolder = RouteCapacity.route_id[index_count]  
                    unpreferredRoutes.append("{}".format(RouteCapacity.agency_id[index_count])+"__"+"{}".format(unprefRouteHolder))
                    print("An unpreferred route has been added. Philita is activated!")
                    # Acquire lock to add one to the Philita counter
                    threadLock.acquire()
                    Philita_count += 1
                    threadLock.release()

        if any(unpreferredRoutes):
#             print("There are "+str(len(unpreferredRoutes))+" unpreferred Routes")

            TripDict = requests.get("http://localhost:{}/otp/routers/default/plan?fromPlace={},{}&toPlace={},{}&time={}&date={}&mode={}&maxWalkDistance={}&arriveBy={}&unpreferredRoutes={}&otherThanPreferredRoutesPenalty={}".format(port_no,journeys.StartLat,journeys.StartLon,journeys.EndLat,journeys.EndLon,T,date,mode,maxWalkDistance,arriveBy,unpreferredRoutes,otherThanPreferredRoutesPenalty))
  
        else:
 
            TripDict = requests.get("http://localhost:{}/otp/routers/default/plan?fromPlace={},{}&toPlace={},{}&time={}&date={}&mode={}&maxWalkDistance={}&arriveBy={}".format(port_no,journeys.StartLat,journeys.StartLon,journeys.EndLat,journeys.EndLon,T,date,mode,maxWalkDistance,arriveBy)) 

            
        # Then recieve the output trip dictionary and parse as JSON
   
        route = TripDict.json()

        
        # Check for a failed route
        try:
   
            itinerary_total = len(route["plan"]["itineraries"])
    
        except:

            itinerary_total = 0

            print("Journey of index "+str(k1)+", "+str(T)+" does not return any itineraries")        
           
        P = 0
        h = 0

        

        while P < itinerary_total:
            leg_total = len(route["plan"]["itineraries"][P]["legs"])
            h = 0
            while h < leg_total:
                try:                                                                           # storing the business for routes
                    unclean_routeId = route["plan"]["itineraries"][P]["legs"][h]["routeId"] 
                    routeId_timeIndex = routesearchfunc(unclean_routeId[2:],T_mod)
                    routeIds_holder.append(routeId_timeIndex)
                    # The threadlock shouldnt be necessary as we are indexing unique locations each time
                    threadLock.acquire()
                    RouteCapacity.loc[routeId_timeIndex,"business"] += 1   
                    threadLock.release()

#                     for step in holding: 
#                         try:
#                             unclean_stopId = route["plan"]["itineraries"][P]["legs"][h][step]["stopId"] # """----------------------------->>>>>>>>>>>>>>>> Here all the itineraries are being added to the stop capacity table. Needs to be taken into account when calculating the number of journeys we are modelling"""
#                             stopId_timeIndex = stopsearchfunc(unclean_stopId[2:],T_mod) # ------------->>>>>>>>>>>> execute the stop search function
#                             stopIds_holder.append(stopId_timeIndex) # ----------------->>>>>>>>>>>>> append the result to the holder
#                             # This shouldnt be necessary as we are indexing unique locations each time
#                             threadLock.acquire()
#                             StopCapacity.loc[stopId_timeIndex,"business"] += 1 
#                             threadLock.release()

#                             h += 1
#                         except:
#                             h += 1                 
#                 except:
#                     None
                except:
                    None

                finally:
                    for step in holding: 
                        try:
                            unclean_stopId = route["plan"]["itineraries"][P]["legs"][h][step]["stopId"] # """----------------------------->>>>>>>>>>>>>>>> Here all the itineraries are being added to the stop capacity table. Needs to be taken into account when calculating the number of journeys we are modelling"""
                            clean_stopId = unclean_stopId[2:] # ---------------->>>>>>>>>>> clean the stopId by removing the first two characters
                            stopId_timeIndex = stopsearchfunc(clean_stopId,T_mod) # ------------->>>>>>>>>>>> execute the stop search function
                            stopIds_holder.append(stopId_timeIndex) # ----------------->>>>>>>>>>>>> append the result to the holder
                            # The threadlock shouldnt be necessary as we are indexing unique locations each time
                            threadLock.acquire()
                            StopCapacity.loc[stopId_timeIndex,"business"] += 1
                            threadLock.release()

                            h += 1
                        except:
                            h += 1
            P += 1

>### The main execution path
>>#### Currently parallelised with 3 instances (24GB)
>>>##### The OTP build will not work well with the current GTFS data - The ATOC and L. datasets are unsynced and out of date
>>>##### As a result routing options will be highly limited - only some of the L. data is within date bounds. 

In [12]:
import threading 
import sys
import pandas as pd
import requests
import time
import psutil
import subprocess
import numpy as np
import re
import math

t_start = time.time()

# Increase the size of the display. Just for me :)
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

""" Meat and Podaydoes """

# Print system version
print(sys.version)

# Define the names we are searching for. Can be taken from the Popen commands
collection = ['otp-2.0.0-SNAPSHOT-shaded.jar']

# Reset the PID
PID = []

# Check if OTP is running. If it is store the PID. Is this MS Windows specific? 
for process in psutil.process_iter():
    try:
        for x in process.cmdline():
                if x in collection:
                    print("The OTP has been already been started")
                    PID.append(process.pid)
    except:
        continue 

# Instantiate the OTP if it has not been started already.

try:
    PID[0]
except: 
    server_1 = subprocess.Popen('cd "C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/OpenTripPlanner2/target" & java -Xmx8G -jar otp-2.0.0-SNAPSHOT-shaded.jar --build --port 8080 --securePort 8081 --serve "C:/Users/Lukas/OneDrive/Documents/Philita/Source for OTP" ', shell=True)
    print("Starting the OTP instance 1")
    PID.append(server_1.pid)

    server_2 = subprocess.Popen('cd "C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/OpenTripPlanner2/target" & java -Xmx8G -jar otp-2.0.0-SNAPSHOT-shaded.jar --build --port 9090 --securePort 9091 --serve "C:/Users/Lukas/OneDrive/Documents/Philita/Source for OTP" ', shell=True)
    print("Starting the OTP instance 2")
    PID.append(server_2.pid)

    server_3 = subprocess.Popen('cd "C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/OpenTripPlanner2/target" & java -Xmx8G -jar otp-2.0.0-SNAPSHOT-shaded.jar --build --port 10010 --securePort 10011 --serve "C:/Users/Lukas/OneDrive/Documents/Philita/Source for OTP" ', shell=True)
    print("Starting the OTP instance 3")
    PID.append(server_3.pid)

# Required
i = 0    

# Wait until server 1 is built and launched
while i < 50:
    try:
        r = requests.get('http://localhost:8080/')
        if r:
            print("server1 has launched correctly")
            i = 9999
        else: 
            print("server1 has not started correctly")
            time.sleep(10)
            i += 1            
    except:
        print("server1 is not reachable")
        time.sleep(10)
        i += 1  

if i == 49:
    print("server 1 has not started in 500 seconds. ERROR 001")
    sys.exit()

i = 0    

# Wait until server 2 is built and launched
while i < 10:
    try:
        r = requests.get('http://localhost:9090/')
        if r:
            print("server2 has launched correctly")
            i = 9999
        else: 
            print("server2 has not started correctly")
            time.sleep(10)
            i += 1            
    except:
        print("server2 is not reachable")
        time.sleep(10)
        i += 1  

if i == 9:
    print("server 2 has not started in 600 seconds. ERROR 001")
    sys.exit()

i = 0

# Wait until server 3 is built and launched
while i < 10:
    try:
        r = requests.get('http://localhost:10010/')
        if r:
            print("server3 has launched correctly")
            i = 9999
        else: 
            print("server3 has not started correctly")
            time.sleep(10)
            i += 1            
    except:
        print("server3 is not reachable")
        time.sleep(10)
        i += 1  

if i == 9:
    print("server 3 has not started in 700 seconds. ERROR 001")
    sys.exit()   
        
# Read variables as a Pandas dataframe
PopInput = pd.read_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/lociLatLong10E2.csv", index_col=0, dtype={"Time":str});  


# Import the stop capacity csv 
StopCapacity = pd.read_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/stops_L.csv", 
                           dtype={"business":np.int32},
                           na_filter=False)

# Import the route capacity csv
RouteCapacity = pd.read_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/routes_L.csv",
                           dtype={"business":np.int32},
                           na_filter=False)

# Route Relative Capacity
StopRelativeBusiness =  pd.read_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/stationcapacity.csv")

# Import the two identifiers csvs
Identifiers_stop_sort = pd.read_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/IdentifiersStopSort.csv")
Identifiers_route_sort = pd.read_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/IdentifiersRouteSort.csv")
Identifiers = pd.read_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/identifiers.csv")

# Create the series indexes
# for the index search
myseries_stopId = pd.Series(StopCapacity["stop_id"], index=range(1,StopCapacity.shape[0]))
myseries_routeId = pd.Series(RouteCapacity["route_id"], index=range(1,RouteCapacity.shape[0]))

# for the identifier search
identifier_indexer_stop = pd.Series(Identifiers_stop_sort["stopId"], index=range(1,Identifiers.shape[0]+1))
identifier_indexer_route = pd.Series(Identifiers_route_sort["routeId"], index=range(1,Identifiers.shape[0]+1))


# Create variables
holding = ["to", "from"]
stop_loading = []
route_loading = []
columns = PopInput.shape[1]
s1 = slice(3,5,1)
stopIds_holder = []
routeIds_holder = []
agencyId_stopId = []
Philita_count = 0
route_b_threshold = 50 # --------------->>>>>>>>> for 10e5

# The penalty for taking routes that are other than preferred. In seconds of penalty. 1200 = 20 minutes
otherThanPreferredRoutesPenalty = 600

# List of all the time intervals
time_list = ["00:00:00", "00:15:00", "00:30:00", "00:45:00", "01:00:00", "01:15:00", "01:30:00", "01:45:00", "02:00:00", "02:15:00", "02:30:00", "02:45:00", "03:00:00", "03:15:00", "03:30:00", "03:45:00", "04:00:00", "04:15:00", "04:30:00", "04:45:00", "05:00:00", "05:15:00", "05:30:00", "05:45:00", "06:00:00", "06:15:00", "06:30:00", "06:45:00", "07:00:00",
        "07:15:00", "07:30:00", "07:45:00", "08:00:00", "08:15:00", "08:30:00", "08:45:00", "09:00:00", "09:15:00", "09:30:00", "09:45:00", "10:00:00", "10:15:00", "10:30:00", "10:45:00", "11:00:00", "11:15:00", "11:30:00", "11:45:00", "12:00:00", "12:15:00", "12:30:00", "12:45:00", "13:00:00", "13:15:00", "13:30:00", "13:45:00", "14:00:00", "14:15:00",
        "14:30:00", "14:45:00", "15:00:00", "15:15:00", "15:30:00", "15:45:00", "16:00:00", "16:15:00", "16:30:00", "16:45:00", "17:00:00", "17:15:00", "17:30:00", "17:45:00", "18:00:00", "18:15:00", "18:30:00", "18:45:00", "19:00:00", "19:15:00", "19:30:00", "19:45:00", "20:00:00", "20:15:00", "20:30:00", "20:45:00", "21:00:00", "21:15:00", "21:30:00",
        "21:45:00", "22:00:00", "22:15:00", "22:30:00", "22:45:00", "23:00:00", "23:15:00", "23:30:00", "23:45:00"] 

# Split up the list of time intervals for parallelisation. 
time_list1 = time_list[0:31]
time_list2 = time_list[31:50]
time_list3 = time_list[50:96]


# Shared request variables
mode = "TRANSIT,WALK" # Transport modes to consider, in this case a combination of walking and transit
maxWalkDistance = "1000" # The maximum distance in meters that you are willing to walk
date = "06-15-2020" # The desired departure date. M-D-Y format
arriveBy = "false"  # Specifies that the given time is when we plan to depart rather than when we want to arrive
row_num = 1

####################################################
print("Processing...")    

t0 = time.time()

threadLock = threading.Lock()

class myThread (threading.Thread):
    def __init__(self, time_list_local, thread_no, port_no, StopCapacity, RouteCapacity):
        threading.Thread.__init__(self)
        self.time_list_local = time_list_local
        self.thread_no = thread_no
        self.port_no = port_no
        self.StopCapacity = StopCapacity
        self.RouteCapacity = RouteCapacity

    def run(self):
        print("Starting "+str(self.thread_no))
        for T1 in self.time_list_local:
            k1 = 0
            while k1 < columns:
                StopCapPhilitaMain_thread(k1, T1, self.port_no, self.StopCapacity, self.RouteCapacity)
                k1 += 1
        print("Exiting " + str(self.thread_no))


# Create new threads
thread1 = myThread(time_list1, 1, 8080, StopCapacity, RouteCapacity)
thread2 = myThread(time_list2, 2, 9090, StopCapacity, RouteCapacity)
thread3 = myThread(time_list3, 3, 10010, StopCapacity, RouteCapacity)

# Start new Threads
thread1.start()
thread2.start()
thread3.start()

thread1.join()
thread2.join()
thread3.join()
print("\nExiting the threading section!")

t1 = time.time()    

print("Adding business for identified routes/stops into the dataframes")

t2 = time.time()

# Update the dataframes with the business
for ID in stopIds_holder:
            StopCapacity.iloc[ID,7] = StopCapacity.iloc[ID,7] + 1

for ID in routeIds_holder:
            RouteCapacity.iloc[ID,8] = RouteCapacity.iloc[ID,8] + 1 
        

t3 = time.time()

# Save the pd.DataFrames into CSVs. 
StopCapacity.to_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/stop_capacity_PHILITA_10e2.csv",index=False) 
RouteCapacity.to_csv("C:/Users/Lukas/OneDrive - University of Bristol/Philita folder number 1 million/Philita/Source for OTP/route_capacity_PHILITA_10e2.csv",index=False)
    
    
# Whether we want to kill the OTP instances upon finishing the script
Kill = True     
    
# If the kill check is true kill the process
if Kill:
    if PID is not None:
        subprocess.run("taskkill  /F  /PID  {}".format(PID))
        print("The OTP has been shut down")
t_final = time.time()
# print(t1-t0)
print("Finished")
print("Journey generation time is "+str(t1-t0)+"s")
print("Appending time is "+str(t3-t2)+"s")
print("Total time is "+str(t_final-t_start)+"s")
print("The number of Philita journey alterations was "+str(Philita_count))

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
Starting the OTP instance 1
Starting the OTP instance 2
Starting the OTP instance 3
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable
server1 is not reachable

C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processing...
Starting 1
Journey from port 8080 with parameters 51.48991 0.06919, 51.47184Starting 2
Journey from port 9090 with parameters 51.54701 -0.04236, 51.58326  -0.226455
Starting 3
Journey from port 10010 with parameters 51.57551 0.090052, 51.49938-0.45536
 -0.134421


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\urllib3\connectionpool.py", line 394, in _make_request
    conn.request(meth

    r = adapter.send(request, **kwargs)
  File "C:\Users\Lukas\anaconda3\envs\py38\lib\site-packages\requests\adapters.py", line 516, in send
    raise ConnectionError(e, request=request)
requests.exceptions.ConnectionError: HTTPConnectionPool(host='localhost', port=10010): Max retries exceeded with url: /otp/routers/default/plan?fromPlace=51.57551,0.090052&toPlace=51.49938,-0.134421&time=12:30:00&date=06-15-2020&mode=TRANSIT,WALK&maxWalkDistance=1000&arriveBy=false (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000019CF35711C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))



Exiting the threading section!
Adding business for identified routes/stops into the dataframes
The OTP has been shut down
Finished
Journey generation time is 4.264998197555542s
Appending time is 0.0s
Total time is 996.4634456634521s
The number of Philita journey alterations was 0
